<a href="https://colab.research.google.com/github/mercygeo/AireTesis/blob/master/Regresion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**PEC 3**

# Regresion: predecir PM25

Este notebook obtiene los datos calidad del aire que están disponibles para múltiples estaciones de medición de aire que acumulan niveles de contaminantes cada hora. Conjunto de datos:Datos en tiempo real [Calidad del aire](https://datos.madrid.es/portal/site/egob/menuitem.c05c1f754a33a9fbe4b2e4b284f1a5a0/?vgnextoid=41e01e007c9db410VgnVCM2000000c205a0aRCRD&vgnextchannel=374512b9ace9f310VgnVCM100000171f5a0aRCRD&vgnextfmt=default) y construye un modelo para predecir el contaminante PM2.5. 
Para hacer esto, se le va a proveer al modelo con la descripcion de varios contaminantes para un periodo de tiempo. La descripcion incluye contaminantes como: NO2, CO2, NO3.

In [0]:
# Use seaborn for pairplot
#!pip install seaborn

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

import pathlib

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

print(tf.__version__)

## El conjunto de datos de calidad del aire

El conjunto de datos esta disponible en Portal de datos abiertos del Ayuntamiento de Madrid [Conjunto de datos calidad del aire](https://datos.madrid.es/portal/site/egob/menuitem.c05c1f754a33a9fbe4b2e4b284f1a5a0/?vgnextoid=41e01e007c9db410VgnVCM2000000c205a0aRCRD&vgnextchannel=374512b9ace9f310VgnVCM100000171f5a0aRCRD&vgnextfmt=default).
Se realizo el preprocesamiento de datos el código esta disponible en el repositorio de la tesis en [GitHub](https://github.com/mercygeo/AireTesis)


### Obtener los datos
Se van a cargar los datos pre-procesados de contaminantes del aire desde google sheets

In [0]:
from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

c = gspread.authorize(GoogleCredentials.get_application_default())


2.   Importar el conjunto de datos usando pandas



In [0]:
# Open our new sheet and read some data.
worksheet = c.open('datos_transformados').sheet1

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()

# Convert to a DataFrame and render.
import pandas as pd
columnas_aire=['FECHA', 'BEN',  'CH4',  'CO',  'EBE','NMHC',  'NO',  'NO2',  'NOx',  'O3', 'PM10',  'PM25',  'SO2',  'TCH',  'TOL']
datos_aire_diarios = pd.DataFrame.from_records(rows, columns =columnas_aire)
datos_aire_diarios[columnas_aire[1:17]] = datos_aire_diarios[columnas_aire[1:17]].apply(pd.to_numeric) 
datos_aire_diarios.tail()

Grafico del PM25 por horario

In [0]:
sns.lineplot(x="CH4", y="PM25", data=datos_aire_diarios, color="indianred")

In [0]:
# Open our new sheet and read some data.
worksheet = c.open('clima_diario_procesado').sheet1

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()

# Convert to a DataFrame and render.
import pandas as pd
columnas_clima=['FECHA','PRE','TEMP',	'VEL']
datos_clima_diario = pd.DataFrame.from_records(rows, columns =columnas_clima)
datos_clima_diario[columnas_clima[1:4]] = datos_clima_diario[columnas_clima[1:4]].apply(pd.to_numeric) 
datos_clima_diario.tail()

In [0]:
sns.lineplot(x="TEMP", y="PRE", data=datos_clima_diario, color="indianred")

### Dividir los conjuntos de datos en train y test

Se va a dividir el conjunto de datos en training y test. Se va a usar el conjunto de test para la evaluación final del modelo

In [0]:
train_aire_dataset = datos_aire_diarios[columnas_aire[1:17]].sample(frac=0.8,random_state=0)
test_aire_dataset = datos_aire_diarios[columnas_aire[1:17]].drop(train_aire_dataset.index)

train_clima_dataset = datos_clima_diario[columnas_clima[1:4]].sample(frac=0.8,random_state=0)
test_clima_dataset = datos_clima_diario[columnas_clima[1:4]].drop(train_clima_dataset.index)

In [0]:
print(train_aire_dataset.shape)
print(train_clima_dataset.shape)

### Inspeccionar los datos

Visualizar la distribución conjunta de algunos pares de columnas del conjunto de entrenamiento.

In [0]:
sns.pairplot(train_aire_dataset [['NO2',  'O3', 'PM10',  'PM25']] )

In [0]:
sns.pairplot(train_clima_dataset [['TEMP',  'VEL', 'PRE']] )

Estadísticas generales:

In [0]:
train_aire_stats = train_aire_dataset.describe()
train_aire_stats.pop("PM25")
train_aire_stats = train_aire_stats.transpose()
train_aire_stats

### Dividir características de las etiquetas

Se va a entrenar el modelo para predecir PM25.

In [0]:
train_aire_labels = train_aire_dataset.pop('PM25')
test_aire_labels = test_aire_dataset.pop('PM25')

In [0]:
print(train_aire_labels.shape)
print(test_aire_labels.shape)

### Normalizar los datos.

Hay que normalizar el conjunto de datos de prueba puesto que los datos tienen diferentes rangos como se puede observar en la tabla de estadisticas generales. Se van a utilizar diferentes tipos de normalización para hacer experimentos y analizar cual devuelve mejores resultados.

In [0]:
from sklearn.preprocessing import Normalizer

#datos del aire
scaler = Normalizer().fit(train_aire_dataset)
normed_train_aire_data = pd.DataFrame(scaler.transform(train_aire_dataset),columns=train_aire_dataset.columns)
scaler = Normalizer().fit(train_aire_dataset)
normed_test_aire_data = pd.DataFrame(scaler.transform(test_aire_dataset),columns=test_aire_dataset.columns)

#datos del clima
scaler = Normalizer().fit(train_clima_dataset)
normed_train_clima_data = pd.DataFrame(scaler.transform(train_clima_dataset),columns=train_clima_dataset.columns)
scaler = Normalizer().fit(train_clima_dataset)
normed_test_clima_data = pd.DataFrame(scaler.transform(test_clima_dataset),columns=test_clima_dataset.columns)

## El modelo

### Guardar y restaurar los modelos
Los diferentes modelos probados se van a  guardar utilizando las librerias proporcionadas por Tensorflow. El codigo a continuacion importa las librerias necesarias.

In [0]:
#!pip install h5py pyyaml
#!pip install tf_nightly

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
!ls '/content/gdrive/My Drive/Colab Notebooks/Tesis/Modelos'

### Construir el modelo

Aquí, usaremos un modelo `Secuencial 'con dos capas ocultas conectadas densamente y una capa de salida que devuelve un valor único y continuo. Los pasos de construcción del modelo están envueltos en una función, `build_model`, para crear varios modelos que luego van a ser utilizados en el ensamblaje.

In [0]:
#[len(train_dataset.keys())]
def build_model(dim):
  model = keras.Sequential([
    layers.Dense(16, activation=tf.nn.relu, input_shape=dim),
    layers.Dense(16, activation=tf.nn.relu),
    layers.Dense(1, activation=tf.nn.elu)
  ])
  
  optimizer = tf.keras.optimizers.RMSprop(0.001)
  
  model.compile(loss='mean_squared_error',
                optimizer=optimizer,
                metrics=['mean_absolute_error', 'mean_squared_error'])
  return model

In [0]:
model_aire = build_model([len(train_aire_dataset.keys())])
model_clima = build_model([len(train_clima_dataset.keys())])

* Se configura la parada  temprana para evitar el sobreajuste.

In [0]:
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

In [0]:
combinedInput = tf.keras.layers.concatenate([model_aire.output, model_clima.output])
 

x = layers.Dense(4, activation="relu")(combinedInput)
x = layers.Dense(1, activation="linear")(x)
 
# our final model will accept categorical/numerical data on the MLP
# input and images on the CNN input, outputting a single value (the
# predicted price of the house)
model = tf.keras.Model(inputs=[model_aire.input, model_clima.input], outputs=x)

La funcion plot_history se usara para mostrar los graficos del error al entrenar el modelo

In [0]:
def plot_history(history):
  hist = pd.DataFrame(history.history)
  hist['epoch'] = history.epoch

  plt.figure()
  plt.xlabel('Epoch')
  plt.ylabel('Mean Abs Error [PM25]')
  plt.plot(hist['epoch'], hist['mean_absolute_error'],
           label='Train Error')
  plt.plot(hist['epoch'], hist['val_mean_absolute_error'],
           label = 'Val Error')
  plt.ylim([0,5])
  plt.legend()

  plt.figure()
  plt.xlabel('Epoch')
  plt.ylabel('Mean Square Error [$PM25^2$]')
  plt.plot(hist['epoch'], hist['mean_squared_error'],
           label='Train Error')
  plt.plot(hist['epoch'], hist['val_mean_squared_error'],
           label = 'Val Error')
  plt.ylim([0,20])
  plt.legend()
  plt.show()

### Inspeccionar el modelo

Usando el método `.summary` se imprime la descripción simple del modelo

In [0]:
model.summary()

Compilar el modelo que usa las dos entradas


In [0]:
#optimizer = tf.keras.optimizers.Adam(1e-4)
#optimizer = tf.train.RMSPropOptimizer(learning_rate=0.001, decay=0.999)
optimizer = tf.keras.optimizers.RMSprop(0.001)

model.compile(loss='mean_squared_error',
                optimizer=optimizer,
                metrics=['mean_absolute_error', 'mean_squared_error'])

### Entrena al modelo

El modelo se va a entrenar durante 1000 épocas y registre la precisión de entrenamiento y validación en el objeto `history`.

In [0]:
checkpoint_path = "/content/gdrive/My Drive/Colab Notebooks/Tesis/Modelos/cpnormDataelu3.ckpt"

#Se pueden guardar checkpoints durante el entrenamiento.
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)


# Display training progress by printing a single dot for each completed epoch
class PrintDot(keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs):
    if epoch % 100 == 0: print('')
    print('.', end='')

EPOCHS = 1000

history = model.fit([normed_train_aire_data, normed_train_clima_data], train_aire_labels, 
                    validation_data=([normed_test_aire_data, normed_test_clima_data], test_aire_labels), 
                    epochs=EPOCHS,
                    verbose=0, 
                    callbacks=[early_stop, PrintDot(), cp_callback])

#Se pueden guardar el modelo completo
model.save('/content/gdrive/My Drive/Colab Notebooks/Tesis/Modelos/normDataelu3.h5')

In [0]:
plot_history(history)

Visualice el progreso del entrenamiento del modelo utilizando las estadísticas almacenadas en el objeto `history`.

In [0]:
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail()

El gráfico muestra que en el conjunto de validación, el error promedio es generalmente alrededor de +/- 1.8 para PM25.

In [0]:
loss, mae, mse = model.evaluate([normed_test_aire_data, normed_test_clima_data], test_aire_labels, verbose=0)

print("Testing set Mean Abs Error: {:5.2f} PM25".format(mae))

### Hacer predicciones

Finalmente, predice los valores de PM25 utilizando datos en el conjunto de pruebas:

In [0]:
test_predictions = model.predict([normed_test_aire_data, normed_test_clima_data]).flatten()

plt.scatter(test_aire_labels, test_predictions)
plt.xlabel('True Values [PM25]')
plt.ylabel('Predictions [PM25]')
plt.axis('equal')
plt.axis('square')
plt.xlim([0,plt.xlim()[1]])
plt.ylim([0,plt.ylim()[1]])
_ = plt.plot([-100, 100], [-100, 100])


Parece que nuestro modelo predice razonablemente bien. Echemos un vistazo a la distribución de errores.

In [0]:
error = test_predictions - test_aire_labels
plt.hist(error, bins = 25)
plt.xlabel("Prediction Error [PM25]")
_ = plt.ylabel("Count")

In [0]:
import requests
import io
url = "http://www.mambiente.munimadrid.es/opendata/horario.csv"

urlData = requests.get(url).content
datos_aire_horarios = pd.read_csv(io.StringIO(urlData.decode('utf-8')), sep=";")
datos_aire_horarios.tail()

In [0]:
# Open our new sheet and read some data.
worksheet = c.open('contaminante_clave').sheet1

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()

# Convert to a DataFrame and render.
import pandas as pd
columnas=['MAGNITUD', 'CONTAMINANTE']
contaminante_clave = pd.DataFrame.from_records(rows, columns =columnas)
contaminante_clave[columnas[0]] = contaminante_clave[columnas[0]].apply(pd.to_numeric) 
contaminante_clave.tail()

In [0]:
header_list = list(datos_aire_horarios)
columns_name = header_list[1:8]
columns_name.append("Valido")
columns_name.append("Hora")
columns_name.append("Valor")

datos_aire_horarios_pro= pd.DataFrame(columns=columns_name)

for i in range(1,24):    
    lista = header_list[1:8]
    lista.append( "V" + ("0"+str(i))[-2:])
    hora ="H" + ("0"+str(i))[-2:]
    datos_aire_horarios1 = pd.melt(datos_aire_horarios, id_vars=lista, value_vars=hora )
    datos_aire_horarios1.columns = columns_name    
    datos_aire_horarios_pro = datos_aire_horarios_pro.append(datos_aire_horarios1)

datos_aire_horarios_pro = datos_aire_horarios_pro[(datos_aire_horarios_pro.Valido =='V')]
datos_aire_horarios_pro.tail()

In [0]:
datos_aire_horarios_pro = pd.merge(datos_aire_horarios_pro, contaminante_clave , left_on='MAGNITUD', right_on='MAGNITUD', copy=False)
datos_aire_horarios_pro.drop("MAGNITUD", axis=1, inplace=True)

In [0]:
datos_aire_horarios_pro1 = pd.pivot_table(datos_aire_horarios_pro, columns='CONTAMINANTE', values=['Valor'], index=['Hora'], aggfunc=np.mean).reset_index()
datos_aire_horarios_pro1

list(datos_aire_horarios_pro1)

flattened = pd.DataFrame(datos_aire_horarios_pro1.to_records(index=False))

columnas= ['HORA', 'BEN',  'CH4',  'CO',  'EBE',
                       'NMHC',  'NO',  'NO2',  'NOx',  'O3',
                         'PM10',  'PM25',  'SO2',  'TCH',  'TOL'] 

flattened.columns = columnas

flattened.tail()

In [0]:
train_horario_dataset = flattened[columnas[1:15]].sample(frac=0.8,random_state=0)
train_horario_labels = train_horario_dataset.pop('PM25')

In [0]:
#datos del aire tiempo real
scaler = Normalizer().fit(train_horario_dataset)
normed_train_horario_data = pd.DataFrame(scaler.transform(train_horario_dataset),columns=train_horario_dataset.columns)

In [0]:
AUX=datos_clima_diario[pd.DatetimeIndex(datos_clima_diario['FECHA']).month==5]
AUX=AUX[1:19]
AUX=AUX[columnas_clima[1:4]]

scaler = Normalizer().fit(AUX)
normed_train_horario_clima = pd.DataFrame(scaler.transform(AUX),columns=AUX.columns)

In [0]:
test_predictions = model.predict([normed_train_horario_data, normed_train_horario_clima]).flatten()
plt.scatter(train_horario_labels, test_predictions)
plt.xlabel('True Values [PM25]')
plt.ylabel('Predictions [PM25]')
plt.axis('equal')
plt.axis('square')
plt.xlim([0,plt.xlim()[1]])
plt.ylim([0,plt.ylim()[1]])
_ = plt.plot([-100, 100], [-100, 100])


In [0]:
error = test_predictions - train_horario_labels
plt.hist(error, bins = 25)
plt.xlabel("Prediction Error [PM25]")
_ = plt.ylabel("Count")